<div style="background: black; padding: 10px 250px"><img src="https://www.veldikompetens.se/wp-content/themes/consid/static/icons/VeldiKompetens_Logo_Web_Negative.svg" title="Veldi kompetens" /></div>

<hr><h1><center>Exercise 3b - Editing and updating your dataset</center></h1>

<h3>Instructions </h3>

<p>In this exercise you will learn more about editing and updating your dataset and we will also perform operations such as cleaning and removing data. We will also get a better overview of the dataset we are working with.  </p>

<h3> 1. Setup </h3>

Authors note: I actually encountered a very interesting issue initially working with this dataset. Pandas has a standard of setting empty inputs in the csv file to NaN (Not a Number) and is the traditional approach of working with missing data. If something is missing by default set it to NaN. However! if you remove the na_values below you will see that bean type is still just an empty string. To fix this I started by reseraching into the csv file and found that instead of the usual format col1val,col2val,col3val,,col5val where there is nothing between col3 and col5 there was instead a blank space i.e col3, ,col5. Upon further examining what this blank space represented it appears to be a special type of encoding for blankspaces namely \xao0 (read some about it if you want to! Very interesting issue.) It can however be solved with the na_values parameter below. This is how it is working with data in real life! Things are never perfect:)

(na_values is a parameter that tells pandas what to identify as NaN values)

Neccesary libraries and the dataset:

In [1]:
import numpy as np
import pandas as pd

# Remeber you need to have the file accessible locally
df = pd.read_csv("flavors_of_cacao.csv", na_values = [" ", u'\xa0'])
df

,Company(Maker-if known),Specific Bean Origin or Bar Name,REF,Review Date,Cocoa Percent,Company Location,Rating,Bean Type,Broad Bean Origin
0,A. Morin,Agua Grande,1876.0,2016.0,63%,France,3.75,NaN,Sao Tome
1,A. Morin,Kpime,1676.0,2015.0,70%,France,2.75,NaN,Togo
2,A. Morin,Atsane,1676.0,2015.0,70%,France,3.00,NaN,Togo
3,A. Morin,Akata,1680.0,2015.0,70%,France,3.50,NaN,Togo
4,A. Morin,Quilla,1704.0,2015.0,70%,France,3.50,NaN,Peru
...,...,...,...,...,...,...,...,...,...
1794,Zotter,Peru,647.0,2011.0,70%,Austria,3.75,NaN,Peru
1795,Zotter,Congo,749.0,2011.0,65%,Austria,3.00,Forastero,Congo
1796,Zotter,Kerala State,749.0,2011.0,65%,Austria,3.50,Forastero,India
1797,Zotter,Kerala State,781.0,2011.0,62%,Austria,3.25,NaN,India


<h3> 2. Examining our data</h3>
<h4> 2.1 Cleaning the data </h4>
<p>As can be seen in the bean type column there are missing values, logically this seems odd right? It might be considered an important column for possible evaluations on what gives a particular chocolate a good rating. Further discussion on this topic will be presented in exercise 3b, but for now  we are going to work with something called cleaning data and how we can do it in pandas. Essentially cleaning data is removing invalid samples from the dataset and the goal of cleaning data can be summarized as achieving the points below. </p>
<ul>
    <li>Is there any row that has no value in any column but still exist as an entry? If so delete.</li>
    <li>Remove samples which are extreme outliers and does not really make any logical sense</li>
    <li>Remove duplicates (Of course depends on data, sometimes duplicates are part of the structure)</li>
</ul>
<p> Lets check out the bean type column which from the getgo seems pretty scarce in data  and evaluate how many of the samples have NaN values</p>

In [2]:
# Some useful things to have access to
samples = len(df)
columns = len(df.columns)


In [ ]:
# Note: Whilst you definitely can do a forloop use pandas/numpy instead! They are much faster and a better practice
bean_col = df["Bean Type"]
samples_missing_bean = bean_col.isna().sum()

print(f"Samples missing bean type value: {samples_missing_bean/samples * 100}%")

Actually, lets perform this check for our entire dataset, just to get a better feel for the data. Usually you would do this much smoother and print less, but for educations sake we go with the more pedalogical approach

In [ ]:
def count_nan():

    for col in df.columns:
        column_name = col
        working_col = df[col]
    
        working_col_missing_bean = working_col.isna().sum()
    
        print(f"The column {column_name} is missing {working_col_missing_bean/samples * 100}% of data! ")
        
count_nan()

Incredible how we could get so much information so quickly right? Lets do some more digging.

Lets wait a bit with reasoning about the huge lack of information about bean type, for now lets check if there are rows that are entirely empty and remove those by the looks of it there might be some

In [ ]:
# This might seem a bit magical, break it down into several lines if you wish!
# Essentially df.isna() checks if there is a NaN value and .all(axis=1) checks if this is true in each column!
nan_samples = df[df.isna().all(axis=1)]
nan_samples

In [ ]:
# Lets go about removing these instances as they only corrupt the data, notice the indexes are still the same as in the original
nan_indexes = nan_samples.index.values
nan_indexes

In [ ]:
# Now lets remove these from the original
df = df.drop(nan_indexes)
count_nan()

# Note: The above example is a more handson approach, pandas has a builtin-functionality for the same purpose;
# df.dropna()

Awesome! Now lets check for duplicates, pandas has a neat functionality builtin which is called duplicated()
duplicated returns a pandas series with boolean values of true/false if there is more than one occurence of an identical sample. To create a "new" dataframe with only the duplicated values you do as we did above by saying df[]. It creates a new dataframe with only the true values.


In [ ]:
#just to show
df[df.duplicated()]

In [ ]:
duplicate_indexes = df[df.duplicated()].index
df = df.drop(duplicate_indexes)
df

In [ ]:
# Lastly after having removed samples you usually re-index the dataset before proceeding and lets save our edited dataset
df = df.reset_index(drop=True)
df.to_csv("edited_choco.csv", index=False)
#In Pandas there is also a way of counting duplicates which may be very relevant for our case since we have a lot of duplicates in terms of companies lets examine this

In Pandas there is also a way of counting duplicates which may be very relevant for our case since we have a lot of duplicates in terms of for example companies lets examine this. In the below code the functionality pivot_table can be used to output a table of the desired readings, you can do a lot of awesome stuff with this function but this is the most simple example. aggfunc is used to describe what action you want to perform on the column, with "size" being a keyword for counting duplicates. NOTE; NaN values are not included.

In [ ]:
df.pivot_table(columns=["Rating"], aggfunc="size")

<h4>2.3 Time for some Q/A! </h4> 

In [ ]:
# Q1: Drop the below sample from the table it is enough to check that there is one less sample than the code above
drop_index = 394
#TODO; no need to reassign the updated value to the df.



Expected output:

<div class="output_subarea output_html rendered_html output_result" dir="auto"><div>
<style scoped="">
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Company(Maker-if known)</th>
      <th>Specific Bean Origin or Bar Name</th>
      <th>REF</th>
      <th>Review Date</th>
      <th>Cocoa Percent</th>
      <th>Company Location</th>
      <th>Rating</th>
      <th>Bean Type</th>
      <th>Broad Bean Origin</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>A. Morin</td>
      <td>Agua Grande</td>
      <td>1876.0</td>
      <td>2016.0</td>
      <td>63%</td>
      <td>France</td>
      <td>3.75</td>
      <td>NaN</td>
      <td>Sao Tome</td>
    </tr>
    <tr>
      <th>1</th>
      <td>A. Morin</td>
      <td>Kpime</td>
      <td>1676.0</td>
      <td>2015.0</td>
      <td>70%</td>
      <td>France</td>
      <td>2.75</td>
      <td>NaN</td>
      <td>Togo</td>
    </tr>
    <tr>
      <th>2</th>
      <td>A. Morin</td>
      <td>Atsane</td>
      <td>1676.0</td>
      <td>2015.0</td>
      <td>70%</td>
      <td>France</td>
      <td>3.00</td>
      <td>NaN</td>
      <td>Togo</td>
    </tr>
    <tr>
      <th>3</th>
      <td>A. Morin</td>
      <td>Akata</td>
      <td>1680.0</td>
      <td>2015.0</td>
      <td>70%</td>
      <td>France</td>
      <td>3.50</td>
      <td>NaN</td>
      <td>Togo</td>
    </tr>
    <tr>
      <th>4</th>
      <td>A. Morin</td>
      <td>Quilla</td>
      <td>1704.0</td>
      <td>2015.0</td>
      <td>70%</td>
      <td>France</td>
      <td>3.50</td>
      <td>NaN</td>
      <td>Peru</td>
    </tr>
    <tr>
      <th>...</th>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
      <td>...</td>
    </tr>
    <tr>
      <th>1790</th>
      <td>Zotter</td>
      <td>Peru</td>
      <td>647.0</td>
      <td>2011.0</td>
      <td>70%</td>
      <td>Austria</td>
      <td>3.75</td>
      <td>NaN</td>
      <td>Peru</td>
    </tr>
    <tr>
      <th>1791</th>
      <td>Zotter</td>
      <td>Congo</td>
      <td>749.0</td>
      <td>2011.0</td>
      <td>65%</td>
      <td>Austria</td>
      <td>3.00</td>
      <td>Forastero</td>
      <td>Congo</td>
    </tr>
    <tr>
      <th>1792</th>
      <td>Zotter</td>
      <td>Kerala State</td>
      <td>749.0</td>
      <td>2011.0</td>
      <td>65%</td>
      <td>Austria</td>
      <td>3.50</td>
      <td>Forastero</td>
      <td>India</td>
    </tr>
    <tr>
      <th>1793</th>
      <td>Zotter</td>
      <td>Kerala State</td>
      <td>781.0</td>
      <td>2011.0</td>
      <td>62%</td>
      <td>Austria</td>
      <td>3.25</td>
      <td>NaN</td>
      <td>India</td>
    </tr>
    <tr>
      <th>1794</th>
      <td>Zotter</td>
      <td>Brazil, Mitzi Blue</td>
      <td>486.0</td>
      <td>2010.0</td>
      <td>65%</td>
      <td>Austria</td>
      <td>3.00</td>
      <td>NaN</td>
      <td>Brazil</td>
    </tr>
  </tbody>
</table>
<p>1794 rows × 9 columns</p>
</div></div>

In [ ]:
#Q2 Output the amount of duplicates in the company location


Expected output: 
<pre>Company Location
Amsterdam              4
Argentina              9
Australia             49
Austria               26
Belgium               40
Bolivia                2
Brazil                17
Canada               125
Chile                  2
Colombia              23
Costa Rica             9
Czech Republic         1
Denmark               15
Domincan Republic      5
Ecuador               54
Eucador                1
Fiji                   4
Finland                2
France               156
Germany               35
Ghana                  1
Grenada                3
Guatemala             10
Honduras               6
Hungary               22
Iceland                3
India                  1
Ireland                4
Israel                 9
Italy                 63
Japan                 17
Lithuania              6
Madagascar            17
Martinique             1
Mexico                 4
Netherlands            4
New Zealand           17
Niacragua              1
Nicaragua              5
Peru                  17
Philippines            1
Poland                 8
Portugal               3
Puerto Rico            4
Russia                 1
Sao Tome               4
Scotland              10
Singapore              3
South Africa           3
South Korea            5
Spain                 25
St. Lucia              2
Suriname               1
Sweden                 5
Switzerland           38
U.K.                  96
U.S.A.               764
Venezuela             20
Vietnam               11
Wales                  1
dtype: int64</pre>

Thats it for this exercise! In the next exercise we will tackle some other problems related to transforming our data types, noramlizing values and reason a bit about the almost 50% missing bean types:) 